# Implementation of the Weber et al. algae model

Simulation of algae growth in a flow-through system. For details on the model see [original publication](https://www.ncbi.nlm.nih.gov/pubmed/22328269).


## How to use this notebook

- Execute the code cell below (by clicking the "play" button or putting the cursor in the cell and pressing shift+tab)
- After code execution, calling the function ```AQPC_interface()``` will
generate the user interface
- It might be necessary to execute the command twice if the backend is not properly updating
- If the program is idle for some while, the server will disconnect. In that case, go to runtime -> restart runtime to start with a clean sheet

In [ ]:
import ipywidgets as widgets # GUI elements
import numpy as np  # numpy for numerical operations and MATLAB-style matrix handling
import pandas as pd # pandas for data handling and R-style data frames
from scipy import integrate # integrate for solving ODEs
import altair as alt # altair for declarative visualization
import os

# Altair theme
def std_theme():
        return {
            'config': {
                'view': {
                    'height': 200,
                    'width': 250,
                },
                'mark_circle': {
                    'size':100
                },
                'axis':{
                    'grid':False,
                    'labelFontSize':15,
                    'titleFontSize':15,
                    'tickSize':10,
                    'tickWidth':1.5
                },
            }
        }
# register the custom theme under a chosen name",
alt.themes.register('std_theme', std_theme)
# enable the newly registered theme",
alt.themes.enable('std_theme')

# light dependence
def Ifunc(I, I_opt):
    return (I/I_opt)*np.exp(1-(I/I_opt))

# temperature dependence
def Tfunc(T, T_min, T_max, T_opt):
    if T < T_opt:
        T_x = T_min
    else:
        T_x = T_max
    return np.exp(-2.3*np.power((T-T_opt)/(T_x-T_opt), 2))
Tfunc = np.vectorize(Tfunc)

# nutrient dependence
def Qfunc(Q, q_min, A):

    fract = (Q/(q_min*A))-1

    return 1 - np.exp(-np.log2(fract))

# nutrient and quota dependence
def QPfunc(A, Q, P, q_min, q_max, k_s):
    Q_depend = (q_max * A - Q) / (q_max - q_min)
    P_depend = P / (k_s + P)
    return Q_depend * P_depend

# dose-response
def Cfunc(C, slope, EC50):
    return 1 - (1 / (1 + np.exp(-slope * (np.log(C) - np.log(EC50)) )))

# function to solve the AQPC model
def solve_AQPC(tmax = "30", # max time
               D    = "0.5", # dilution rate
               T     = "24",  # temperature
               T_min = "0",  # minimum temperature
               T_max = "35",  # maximum temperature
               T_opt = "27", # optimum temperature
               R0    = "0.36", # nutrient concentration in culture medium
               C_in  = "0.0", # toxicant concentration in fresh medium
               I     = "100", # light intensity
               I_opt = "120",
               mu_max = "1.7380", # max. growth rate
               m_max  = "0.0500", # max. mortality rate
               v_max  = "0.0520", # max. P uptake
               k = ".5",     # half saturation constant for P uptake
               q_min = "0.0011",
               q_max = "0.0144",
               slope = "2",
               EC50  = "150",
               k_s   = "0.0680"):

    """solve the AQPC model, given parameters"""

    # Convert strings to numbers

    tmax  = int(tmax)
    D = float(D)
    T = float(T)
    T_min = float(T_min)
    T_max = float(T_max)
    T_opt = float(T_opt)
    R0    = float(R0)
    C_in  = float(C_in)
    I     = float(I)
    I_opt = float(I_opt)
    mu_max = float(mu_max)
    m_max = float(m_max)
    v_max = float(v_max)
    k = float(k)
    q_min = float(q_min)
    q_max = float(q_max)
    slope = float(slope)
    EC50  = float(EC50)
    k_s   = float(k_s)

    time = np.arange(start=0, stop=tmax, step=0.5)

    fT = Tfunc(T, T_min, T_max, T_opt)
    fI = Ifunc(I, I_opt)

    # define the ODE system
    def AQPC_deriv(AQPC, t0):
        """compute the derivative of the AQPC model"""
        A, Q, P, C = AQPC # unpack state vars

        fQ  = Qfunc(Q, q_min, A)
        fQP = QPfunc(A, Q, P, q_min, q_max, k_s)
        fC  = Cfunc(C, slope, EC50)

        return [((mu_max * fT * fI * fQ * fC) - m_max - D) * A,   # dA
              v_max * fQP * A - (m_max + D) * Q,              # dQ
              D * R0 - D * P + m_max * Q - (v_max * fQP * A), # dP
              C_in * D - k * C - D * C]                       # dC

    # solve the model
    solution = integrate.odeint(AQPC_deriv, (2, .1, 2, 0), time)
    solution = np.nan_to_num(solution)
    # collect data in dataframe
    model_df = pd.DataFrame({
      'tday': time, # time in days
      'A': solution[:,0],   # population density
      'Q': solution[:,1],   # nutrient quota
      'P': solution[:,2],   # (external) phosphorus
      'C': solution[:,3]    # toxicant concentration
    })

    return model_df

out = solve_AQPC()

# split a string by commas, convert individual elements to float and return
# as numpy array
def array_from_text(instring):
  return np.array([float(x) for x in instring.split(sep=",")])

# generate a parameter box (name + value + unit + meaning)
def generate_par_box(name, default, unit, meaning):
    box_layout = widgets.Layout(display="flex",
                              box="solid",
                              color="blue")

    name_widget  = widgets.Text(description=name, value=str(default),
                           layout = box_layout)
    meaning_widget = widgets.Label(value = meaning+" ("+unit+")")

    par_box = widgets.HBox(children = [name_widget, meaning_widget])
    return par_box

my_par_box = generate_par_box(name    = "C_in",
                              default = 0,
                              unit    = "µg/L",
                              meaning = "toxicant concentration in culture medium")

# fgenerate a "parameter collection" (e.g. environmental parameters)
def generate_par_collection(names, defaults, units, meanings):

    children = []

    for i,_ in enumerate(names):
        par_box = generate_par_box(names[i],
                                   defaults[i],
                                   units[i],
                                   meanings[i])
        children.append(par_box)

    par_collection = widgets.VBox(children = children)

    return par_collection

# generate state varible charts in batch mode
def generate_batch_chart(main_df):

    # generate a chart object
    # with time on x axis
    # and chemical concentration indicated by color (treated as nominal variable)
    base = alt.Chart(main_df).mark_line().encode(x="tday:Q", color="C_in:N")

    # add a y axis for every state variable
    At = base.encode(y="A:Q")
    Pt = base.encode(y="P:Q")
    Qt = base.encode(y="Q:Q")
    Ct = base.encode(y="C:Q")

    # concatenate horizontally via pipe operator
    chart = At | Qt | Pt | Ct

    # return chart object
    return chart

def AQPC_interface():

    """ Display the AQPC GUI """

    # generate interface elements for entering environmental parameters
    env_pars = generate_par_collection(
                                 # parmeter names
                                 names    = ["T", "I", "C_in", "tmax", "R0", "D"],
                                 # default values
                                 defaults = [24, 100,  "0, 150, 300", 30, 0.36, 0.5],
                                 # units
                                 units    = ["°C", "µmol/sqm/s", "µg/L", "days", "mg P/L", "/d"],
                                 # short description
                                 meanings  = ["temperature",
                                              "light intensity",
                                              "toxicant concentration",
                                              "max. simulation time",
                                              "nutrient concentration in medium",
                                              "dilution rate"])
    # generate interface elements for entering species-specific parameters
    spec_pars = generate_par_collection(
                            # parameter names
                            names=["mu_max",
                                   "q_min",
                                   "q_max",
                                   "v_max",
                                   "k",
                                   "k_s",
                                   "m_max",
                                   "EC50",
                                   "slope"],
                            # default values
                            defaults = [1.7380,
                                        0.0011,
                                        0.0144,
                                        0.0520,
                                        0.5,
                                        0.0680,
                                        0.0500,
                                        150,
                                        2],
                           # units
                           units    = ["1/d",
                                       "µgP/µg fresh wt",
                                       "µgP/µg fresh wt",
                                       "µgP/µg fresh wt/d",
                                       "1/d",
                                       "mgP/L",
                                       "1/d",
                                       "µg/L",
                                       "-"],
                           # short description
                           meanings = ["maximum growth rate",
                                       "minimum intracellular P",
                                       "maximum intracellular P",
                                       "maximum P uptake rate",
                                       "degradation rate of toxicant",
                                       "half-saturation constant of P uptake",
                                       "background mortality rate",
                                       "median effective concentration",
                                       "slope"])

    # Other interface elements
    # "run" button -> run a simulation
    run_button = widgets.Button(description="Run", tooltip="run simulation with given parameters")
    # "save" checkbox -> determine whether to save data to .csv
    save_checkbox = widgets.Checkbox(
                                value=False,
                                description="Save output",
                                tooltip="Should output be saved? \n (enter output file path below before clicking Run)")
    # "save path" text field -> determine filename of output file
    save_path = widgets.Text(value="output.csv")
    # output widget -> area to display charts
    output = widgets.Output()

    # function that determines what happens when "run" button is clicked
    def on_run_button_clicked(b):

      """ Solve the model when "run" button is clicked """

      # read environmnetal parameters from interface
      T     = env_pars.children[0].children[0].value
      I     = env_pars.children[1].children[0].value
      C_in_sweep  = array_from_text(env_pars.children[2].children[0].value)
      tmax  = env_pars.children[3].children[0].value
      R0    = env_pars.children[4].children[0].value
      D     = env_pars.children[5].children[0].value

      # read species-specific parameters from interface
      mu_max = spec_pars.children[0].children[0].value
      q_min  = spec_pars.children[1].children[0].value
      q_max  = spec_pars.children[2].children[0].value
      v_max  = spec_pars.children[3].children[0].value
      k   = spec_pars.children[4].children[0].value
      k_s = spec_pars.children[5].children[0].value
      m_max = spec_pars.children[6].children[0].value
      EC50  = spec_pars.children[7].children[0].value
      slope = spec_pars.children[8].children[0].value

      # generate an empty dataframe to store results in
      main_df = pd.DataFrame()

      # iterate over toxicant concentrations
      for i,C_in in enumerate(C_in_sweep):

        # solve model and store output in dataframe
        # check solve_AQPC() source code to see how this is done
        model_df = solve_AQPC(T=T, I=I, C_in=C_in, tmax=tmax, R0=R0, D=D,
                              mu_max=mu_max,
                              q_min=q_min,
                              q_max=q_max,
                              v_max=v_max,
                              k=k,
                              k_s=k_s, m_max=m_max,
                            EC50=EC50, slope=slope)

        # add parameter values to model output dataframe
        model_df["T"] = T
        model_df["I"] = I
        model_df["C_in"] = C_in
        model_df["tmax"] = tmax

        model_df["mu_max"] = mu_max
        model_df["q_min"]  = q_min
        model_df["q_max"]  = q_max
        model_df["v_max"]  = v_max
        model_df["k"]  = k
        model_df["k_s"]  = k_s
        model_df["m_max"]  = m_max
        model_df["EC50"]  = EC50
        model_df["slope"]  = slope

        # append results to main output dataframe
        main_df = pd.concat([main_df, model_df])

      # if checkbox is clicked --> write dataframe to .csv
      if save_checkbox.value == True:
          main_df.to_csv(save_path.value)

      # remove previously generated charts
      output.clear_output(wait=True)
      # generate new chart object
      chart = generate_batch_chart(main_df)
      # display chart
      display(chart)

    # assign function on_run_button_clicked to run_button widget
    run_button.on_click(on_run_button_clicked)

    # generate a panel to enter environmental and species-specific parameters
    pars_panel= widgets.HBox(children=[env_pars, spec_pars])
    # generate a panel containing buttons, checkboxes, text_fields...
    buttons_panel = widgets.VBox(children=[save_checkbox,
                                           save_path,
                                           run_button])
    # combine into a single panel
    input_panel = widgets.HBox(children=[
        pars_panel, buttons_panel
    ])

    # output panel contains only chart objects
    output_panel = widgets.HBox(children=[output])

    # combine components into a single interface object
    interface = widgets.VBox(children=[input_panel, output_panel])

    return interface

<ipython-input-1-2ab397efee2a>:29: AltairDeprecationWarning: 
Deprecated since `altair=5.5.0`. Use altair.theme instead.
Most cases require only the following change:

    # Deprecated
    alt.themes.enable('quartz')

    # Updated
    alt.theme.enable('quartz')

If your code registers a theme, make the following change:

    # Deprecated
    def custom_theme():
        return {'height': 400, 'width': 700}
    alt.themes.register('theme_name', custom_theme)
    alt.themes.enable('theme_name')

    # Updated
    @alt.theme.register('theme_name', enable=True)
    def custom_theme():
        return alt.theme.ThemeConfig(
            {'height': 400, 'width': 700}
        )

See the updated User Guide for further details:
    https://altair-viz.github.io/user_guide/api.html#theme
    https://altair-viz.github.io/user_guide/customization.html#chart-themes
  alt.themes.register('std_theme', std_theme)
<ipython-input-1-2ab397efee2a>:61: RuntimeWarning: divide by zero encountered in log
  retur

In [ ]:
1-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------AQPC_interface()

<ipython-input-10-2ab397efee2a>:61: RuntimeWarning: divide by zero encountered in log
  return 1 - (1 / (1 + np.exp(-slope * (np.log(C) - np.log(EC50)) )))
<ipython-input-10-2ab397efee2a>:61: RuntimeWarning: divide by zero encountered in log
  return 1 - (1 / (1 + np.exp(-slope * (np.log(C) - np.log(EC50)) )))
<ipython-input-10-2ab397efee2a>:61: RuntimeWarning: divide by zero encountered in log
  return 1 - (1 / (1 + np.exp(-slope * (np.log(C) - np.log(EC50)) )))


alt.HConcatChart(...)

In [ ]:
#df = pd.read_csv("output.csv")
#df.head()

## Save output to google drive
Execute the code cell below and follow instructions to connect the notebook to your Gdrive:

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

Execute this code cell to save model output to a google drive location:

In [ ]:
output_filename = "model_output.csv"
working_directory = "gdrive/My Drive/Colab Notebooks"
outfile_name = os.path.join(working_directory, output_filename)
outfile = pd.read_csv("output.csv")
outfile.to_csv(outfile_name)

The file should now be saved in the location you have entered for "working directory" and "output_filename".